In [ ]:
!pip install ruptures --quiet

In [ ]:
import mlflow
import pandas as pd
import numpy as np
from datetime import datetime
from pyspark.sql import functions as F, types as T
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.arima.model import ARIMA
import mlflow.statsmodels 
from typing import Optional
from sklearn.linear_model import LinearRegression
import ruptures as rpt


In [ ]:
input_path = ""
holdout_weeks=4
forecast_horizon=52

## Load Demand Data

Review data, statistics and clean

In [ ]:
df = (
    spark.read.option("header", True).option("inferSchema", True).csv(input_path)
    .withColumn("Date", F.to_date(F.col("Date"), "M/d/yyyy")) )

stats = df.describe()
display(stats)

Basic data cleaning and Outlier Removal for Quantity (IQR Method), grouped by Site and Fuel

In [ ]:

from pyspark.sql.functions import col, percentile_approx


# Drop duplicate rows
df_clean = df.dropDuplicates()

# Remove rows with any nulls in key columns ('Site', 'Fuel', 'Quantity')
df_clean = df_clean.dropna(subset=['Site', 'Fuel', 'Quantity'])


# Calculate group-specific Q1 and Q3
iqr_stats = df_clean.groupBy("Site", "Fuel").agg(
    percentile_approx("Quantity", 0.25, 100).alias("Q1"),
    percentile_approx("Quantity", 0.75, 100).alias("Q3")
).withColumn("IQR", col("Q3") - col("Q1")) \
 .withColumn("lower_bound", col("Q1") - 1.5 * col("IQR")) \
 .withColumn("upper_bound", col("Q3") + 1.5 * col("IQR"))

# Join bounds back to original dataframe and apply filtering
df_iqr = df_clean.join(iqr_stats, on=["Site", "Fuel"], how="left")
df_iqr = df_iqr.filter((col("Quantity") >= col("lower_bound")) & (col("Quantity") <= col("upper_bound")))

# Remove IQR calculation columns now
df_iqr = df_iqr.drop("Q1", "Q3", "IQR", "lower_bound", "upper_bound")

# Remove (Site, Fuel) groups with count < 3
group_counts = df_iqr.groupBy("Site", "Fuel").count()
valid_groups = group_counts.filter(col("count") >= 3).select("Site", "Fuel")
df_final = df_iqr.join(valid_groups, on=["Site", "Fuel"], how="inner")

# Show numeric summary statistics
stats = df_final.describe()
display(stats)

Clean-up, order and standardize time series for forecasting

In [ ]:
weekly_df = (
    df_final.withColumn("week_start", F.date_trunc("week", F.col("Date")))
      .groupBy("Site", "Fuel", "week_start")
      .agg(F.sum("Quantity").alias("Quantity"))
)

# min/max range per Site/Fuel
ranges = (
    weekly_df
      .groupBy("Site", "Fuel")
      .agg(
          F.min("week_start").alias("min_week"),
          F.max("week_start").alias("max_week"),
      )
)

# explode a complete weekly sequence (7-day step)
calendar = (
    ranges
      .withColumn(
          "week_start",
          F.explode(F.sequence(F.col("min_week"), F.col("max_week"), F.expr("interval 7 days")))
      )
      .select("Site", "Fuel", "week_start")
)

# left join + fill missing as 0
weekly_full = (
    calendar
      .join(weekly_df, on=["Site", "Fuel", "week_start"], how="left")
      .withColumn("Quantity", F.coalesce(F.col("Quantity"), F.lit(0.0)))
)

weekly_full = (
    weekly_full
      .withColumn("Quantity", F.col("Quantity").cast("double"))
      .orderBy("Site", "Fuel", "week_start")
)


In [ ]:
display(weekly_full)

In [ ]:
def fit_and_forecast_ets(
    pdf: pd.DataFrame,
    *,
    holdout_weeks: int = 6, ## holdout weeks for backtestings
    forecast_horizon: int = 52,
    weekly_season_length: int = 52,
    use_boxcox_for_ets: bool = False,
    decay_rate: float = 0.03,
    floor_percentile: float = 10.0,
    min_floor: float = 10.0,
    fallback_confidence: float = 0.60,
) -> pd.DataFrame:
    """
    Fit a weekly ETS model and generate historical, backtest, and future forecasts
    for a single Site/Fuel time series.

    This implementation is designed for noisy demand data with weak or inconsistent
    seasonality. It prevents forecast collapse to zero by enforcing a data-driven
    lower bound and produces a decaying confidence score for each forecasted point.

    Parameters
    ----------
    pdf : pd.DataFrame
        Input data for a single Site/Fuel combination. Must contain columns:
        ['Site', 'Fuel', 'week_start', 'Quantity'].
    holdout_weeks : int, default 4
        Number of most recent weeks reserved for backtesting.
    forecast_horizon : int, default 24
        Number of future weeks to forecast.
    weekly_season_length : int, default 52
        Seasonal period (weeks). Seasonality is only enabled if at least two full
        cycles are available.
    use_boxcox_for_ets : bool, default False
        Whether to apply a Box–Cox transform when fitting ETS.
    decay_rate : float, default 0.03
        Exponential decay rate applied to confidence scores as forecast horizon
        increases.
    floor_percentile : float, default 10.0
        Percentile of the training data used to compute a lower bound for forecasts.
    min_floor : float, default 10.0
        Absolute minimum floor applied to forecasts (prevents hard-zero collapse).
    fallback_confidence : float, default 0.60
        Base confidence score used when no holdout data is available.

    Returns
    -------
    pd.DataFrame
        Concatenated DataFrame with rows of type:
        - 'actual'   : historical training values
        - 'backtest' : forecasts over the holdout window (if any)
        - 'forecast' : future forecasts

        Columns:
        ['Site', 'Fuel', 'week_start', 'value', 'type', 'method', 'confidence_score']
    """

    site, prod = map(str, (pdf["Site"].iloc[0], pdf["Fuel"].iloc[0]))

    # --- Weekly series
    s = pdf.copy()
    s["week_start"] = pd.to_datetime(s["week_start"], errors="coerce")
    s = s.dropna(subset=["week_start"]).sort_values("week_start")

    ts = (
        s.set_index("week_start")["Quantity"]
        .astype(float)
        .asfreq("7D")
        .interpolate("linear")
        .bfill()
        .ffill()
    )
    if len(ts) < 3 or ts.isna().any():
        raise ValueError(f"Insufficient clean training data for {site}-{prod}: len(ts)={len(ts)}")

    # --- Split
    holdout_weeks = int(max(0, holdout_weeks))
    train_end = max(3, len(ts) - holdout_weeks)
    train, holdout = ts.iloc[:train_end], ts.iloc[train_end:]

    # --- Floor to prevent hard-zero collapse
    floor = max(min_floor, float(np.nanpercentile(train.values, floor_percentile)))

    # -----------------------------
    # Model routing + confidence
    # -----------------------------
    # This series family is highly sensitive to intermittency + regime switches:
    # - mostly zeros with occasional spikes => intermittent (TSB/SBA) Teunter–Syntetos–Babai
    # - switches between inactive/active regimes => gated hybrid
    # - strong annual seasonality => seasonal ETS + season-aware confidence
    EPS = 1e-6
    CONF_MIN, CONF_MAX = 0.10, 0.95

    def _smape(y_true: np.ndarray, y_pred: np.ndarray) -> float:
        denom = np.maximum(np.abs(y_true) + np.abs(y_pred), EPS)
        return float(np.mean(2.0 * np.abs(y_pred - y_true) / denom))  # in [0, 2]

    def _wape(y_true: np.ndarray, y_pred: np.ndarray) -> float:
        denom = max(float(np.sum(np.abs(y_true))), EPS)
        return float(np.sum(np.abs(y_pred - y_true)) / denom)

    def _brier(y_true_occ: np.ndarray, p_occ: np.ndarray) -> float:
        p = np.clip(p_occ, 0.0, 1.0)
        return float(np.mean((p - y_true_occ) ** 2))

    def _signal_strength(train_arr: np.ndarray) -> float:
        # Simple, robust: reward non-zero rate (overall + recent), penalize extreme volatility of non-zero sizes
        nonzero_rate = float(np.mean(train_arr > 0))
        recent = train_arr[-13:] if len(train_arr) >= 13 else train_arr
        recent_activity = float(np.mean(recent > 0))

        nz = train_arr[train_arr > 0]
        if len(nz) >= 5:
            cv = float(np.std(nz) / max(np.mean(nz), EPS))
        else:
            cv = 2.0

        score = (0.6 * nonzero_rate + 0.4 * recent_activity) * float(np.exp(-0.25 * cv))
        return float(np.clip(score, 0.05, 1.0))

    def _regime_stability(train_arr: np.ndarray) -> float:
        # Penalize sudden activity change (classic regime switching)
        if len(train_arr) >= 26:
            a = float(np.mean(train_arr[-13:] > 0))
            b = float(np.mean(train_arr[-26:-13] > 0))
            shift = abs(a - b)
        else:
            shift = 0.0
        return float(np.clip(1.0 - 0.75 * shift, 0.25, 1.0))

    def _decay_conf(base: float, h: np.ndarray) -> np.ndarray:
        return np.clip(base * np.exp(-decay_rate * h.astype(float)), CONF_MIN, CONF_MAX)

    def _base_conf_from_holdout(
        train_arr: np.ndarray,
        hold_arr: np.ndarray,
        pred_arr: np.ndarray,
        *,
        mode: str,
        pred_occ: np.ndarray | None = None,
    ) -> float:
        # Composite base confidence:
        # - fit_quality: error on holdout (metric depends on mode)
        # - signal_strength: how much predictable signal exists
        # - regime_stability: penalize regime transitions
        sig = _signal_strength(train_arr)
        stab = _regime_stability(train_arr)

        if len(hold_arr) == 0:
            base = float(np.clip(fallback_confidence, CONF_MIN, CONF_MAX))
            return base

        if mode == "INTERMITTENT" and pred_occ is not None:
            # Fit_quality for intermittent demand:
            # - size error on non-zero actuals (WAPE on non-zeros)
            # - occurrence calibration (Brier score)
            y_true = hold_arr.astype(float)
            y_pred = pred_arr.astype(float)
            y_occ = (y_true > 0).astype(float)
            b = _brier(y_occ, pred_occ.astype(float))  # 0..1 (lower is better)

            nz = y_true > 0
            if int(nz.sum()) >= 2:
                w = _wape(y_true[nz], y_pred[nz])
                size_quality = float(np.exp(-2.0 * w))
            else:
                size_quality = float(np.exp(-1.5 * _smape(y_true, y_pred)))

            occ_quality = float(np.exp(-2.0 * b))
            fit_quality = float(np.clip(0.6 * occ_quality + 0.4 * size_quality, 0.0, 1.0))
        else:
            # Fit_quality for continuous/seasonal demand:
            y_true = hold_arr.astype(float)
            y_pred = pred_arr.astype(float)
            nz = y_true > 0
            if int(nz.sum()) >= 2:
                w = _wape(y_true[nz], y_pred[nz])
                fit_quality = float(np.exp(-2.0 * w))
            else:
                fit_quality = float(np.exp(-1.5 * _smape(y_true, y_pred)))

        base = fit_quality * sig * stab
        return float(np.clip(base, CONF_MIN, CONF_MAX))

    def _classify(train_arr: np.ndarray) -> str:
        zero_rate = float(np.mean(train_arr == 0))
        recent = train_arr[-13:] if len(train_arr) >= 13 else train_arr
        recent_nonzero = float(np.mean(recent > 0))
        # Intermittent/lumpy 
        if zero_rate > 0.70 and recent_nonzero < 0.30:
            return "INTERMITTENT"
        # Regime switching 
        if len(train_arr) >= 26:
            a = float(np.mean(train_arr[-13:] > 0))
            b = float(np.mean(train_arr[-26:-13] > 0))
            if abs(a - b) >= 0.30 and (0.30 <= zero_rate <= 0.85):
                return "REGIME"
        # Seasonal/continuous 
        return "SEASONAL_OR_CONTINUOUS"

    # --- Intermittent demand model: TSB (preferred) + SBA fallback
    def _fit_tsb(train_arr: np.ndarray, alpha: float = 0.20, beta: float = 0.20):
        # TSB maintains occurrence probability p_t and demand size z_t when it occurs.
        p = 0.10
        z = float(np.mean(train_arr[train_arr > 0])) if np.any(train_arr > 0) else 0.0
        for y in train_arr:
            occ = 1.0 if y > 0 else 0.0
            p = p + alpha * (occ - p)
            if y > 0:
                z = z + beta * (y - z)
        return p, z

    def _tsb_forecast(p: float, z: float, steps: int):
        # Expected demand each step
        yhat = np.full(shape=(steps,), fill_value=(p * z), dtype=float)
        # Occurrence probability (constant under this simple TSB forecast)
        phat = np.full(shape=(steps,), fill_value=p, dtype=float)
        return yhat, phat

    def _croston_sba_forecast(train_arr: np.ndarray, steps: int, alpha: float = 0.10):
        # SBA: Croston with bias correction (1 - alpha/2)
        nz_idx = np.where(train_arr > 0)[0]
        if len(nz_idx) == 0:
            return np.zeros(steps, dtype=float)

        # Initialize
        z = float(train_arr[nz_idx[0]])
        p = float(nz_idx[0] + 1)  # interval length
        last_idx = nz_idx[0]

        for idx in nz_idx[1:]:
            y = float(train_arr[idx])
            interval = float(idx - last_idx)
            z = z + alpha * (y - z)
            p = p + alpha * (interval - p)
            last_idx = idx

        yhat = (z / max(p, EPS)) * (1.0 - alpha / 2.0)
        return np.full(steps, yhat, dtype=float)

    def mk_df(dates, values, typ, method, conf):
        return pd.DataFrame({
            "Site": site,
            "Fuel": prod,
            "week_start": dates,
            "value": values.astype(float),
            "type": typ,
            "method": method,
            "confidence_score": conf,
        })

    def clean(x: pd.Series | np.ndarray, idx=None, *, floor_override: float | None = None) -> pd.Series:
        ser = pd.Series(x, index=idx)
        lo = floor if floor_override is None else float(floor_override)
        return ser.clip(lower=lo).round().astype(int)

    parts = []

    # actuals (train)
    parts.append(mk_df(train.index, train.values, "actual", "N/A", np.nan))

    # --- Choose model family
    train_arr = train.values.astype(float)
    hold_arr = holdout.values.astype(float)
    series_class = _classify(train_arr)

    # --- Forecast holdout + future counts
    h_hold, h_fut = len(holdout), int(forecast_horizon)
    total_steps = h_hold + h_fut

    # --- Default values
    base_conf = float(np.clip(fallback_confidence, CONF_MIN, CONF_MAX))
    method_label = "N/A"
    pred_all = None
    pred_occ_all = None
    floor_override = None  # allow true zeros for intermittent forecasts

    # -----------------------------
    # Intermittent: TSB (do we have exogenous signals)
    # -----------------------------
    if series_class == "INTERMITTENT":
        # NOTE: For intermittent demand, true zeros are meaningful; do not force a positive floor.
        floor_override = 0.0
        p, z = _fit_tsb(train_arr, alpha=0.20, beta=0.20)
        yhat_all, phat_all = _tsb_forecast(p, z, total_steps)
        pred_all = pd.Series(yhat_all, index=pd.RangeIndex(total_steps))
        pred_occ_all = pd.Series(phat_all, index=pd.RangeIndex(total_steps))
        method_label = "TSB"

    # ------------------------------------------
    # Regime switching: gated hybrid
    # ------------------------------------------
    elif series_class == "REGIME":
        # Determine regime from RECENT activity in the training window
        recent = train_arr[-13:] if len(train_arr) >= 13 else train_arr
        recent_activity = float(np.mean(recent > 0))

        if recent_activity < 0.35:
            # Inactive regime -> intermittent
            floor_override = 0.0
            p, z = _fit_tsb(train_arr, alpha=0.20, beta=0.20)
            yhat_all, phat_all = _tsb_forecast(p, z, total_steps)
            pred_all = pd.Series(yhat_all, index=pd.RangeIndex(total_steps))
            pred_occ_all = pd.Series(phat_all, index=pd.RangeIndex(total_steps))
            method_label = "HYBRID_TSB_INACTIVE"
        else:
            # Active regime -> ETS (seasonal only if enough history)
            use_seasonal = len(train) >= 2 * weekly_season_length
            seasonal = "add" if use_seasonal else None
            method_label = "HYBRID_ETS" if use_seasonal else "HYBRID_ETS_NoSeasonality"

            model = ExponentialSmoothing(
                train,
                trend="add",
                damped_trend=True,
                seasonal=seasonal,
                seasonal_periods=(weekly_season_length if use_seasonal else None),
            ).fit(optimized=True, use_boxcox=(use_boxcox_for_ets or None))

            preds = model.forecast(total_steps)
            pred_all = pd.Series(preds.values, index=pd.RangeIndex(total_steps))

    # ------------------------------------------------
    # Seasonal/continuous: seasonal ETS
    # ------------------------------------------------
    else:
        use_seasonal = len(train) >= 2 * weekly_season_length
        seasonal = "add" if use_seasonal else None
        method_label = "ETS" if use_seasonal else "ETS_NoSeasonality"

        model = ExponentialSmoothing(
            train,
            trend="add",
            damped_trend=True,
            seasonal=seasonal,
            seasonal_periods=(weekly_season_length if use_seasonal else None),
        ).fit(optimized=True, use_boxcox=(use_boxcox_for_ets or None))

        preds = model.forecast(total_steps)
        pred_all = pd.Series(preds.values, index=pd.RangeIndex(total_steps))

    # --- Prepare indices
    if h_hold:
        bt_index = holdout.index
    fut_index = pd.date_range(ts.index[-1] + pd.Timedelta(days=7), periods=h_fut, freq="7D")

    # --- Backtest (optional)
    if h_hold:
        bt_raw = pd.Series(pred_all.iloc[:h_hold].values, index=bt_index)

        # Compute base confidence from holdout using the model family
        if series_class == "INTERMITTENT" or (series_class == "REGIME" and method_label.startswith("HYBRID_TSB")):
            bt_occ = pred_occ_all.iloc[:h_hold].values if pred_occ_all is not None else None
            base_conf = _base_conf_from_holdout(
                train_arr=train_arr,
                hold_arr=hold_arr,
                pred_arr=bt_raw.values.astype(float),
                mode="INTERMITTENT",
                pred_occ=bt_occ,
            )
        else:
            base_conf = _base_conf_from_holdout(
                train_arr=train_arr,
                hold_arr=hold_arr,
                pred_arr=bt_raw.values.astype(float),
                mode="CONTINUOUS",
            )

        # Continuous decay across backtest + forecast (no restart)
        h = np.arange(1, h_hold + 1)
        bt_conf = _decay_conf(base_conf, h)

        bt = clean(bt_raw, bt_index, floor_override=floor_override)
        parts.append(mk_df(bt.index, bt.values, "backtest", method_label, bt_conf.astype(float)))

    # --- Forecast
    fut_raw_vals = pred_all.iloc[h_hold:].values.astype(float)
    fut_raw = pd.Series(fut_raw_vals, index=fut_index)
    fut = clean(fut_raw, fut_index, floor_override=floor_override)

    # Continue decay AFTER the holdout window (don't restart at h=1)
    h = np.arange(h_hold + 1, h_hold + h_fut + 1)
    fut_conf = _decay_conf(base_conf, h)

    parts.append(mk_df(fut.index, fut.values, "forecast", method_label, fut_conf.astype(float)))

    return pd.concat(parts, ignore_index=True)


In [ ]:

def fit_and_forecast(
    pdf: pd.DataFrame,
    *,
    holdout_weeks: int = 6,
    forecast_horizon: int = 52,
    decay_rate: float = 0.03,
    floor_percentile: float = 10.0,
    min_floor: float = 10.0,
    fallback_confidence: float = 0.60,
    max_breakpoints: int = 5,
    min_segment_weeks: int = 8,
    #  knobs to prevent “runaway ramps” ---
    min_last_segment_weeks: int = 16,  # require enough history to trust the most-recent slope
    trend_shrink: float = 0.5,         # 0..1 shrink chosen slope toward 0
    slope_cap: float | None = None,    # optional absolute cap on slope (units per week)
    use_dominant_trend: bool = True,   # if False, falls back to last-segment logic
) -> pd.DataFrame:
    """
    Fit a weekly piecewise-linear model with changepoints and produce:
      - historical actuals (train)
      - backtest predictions over the holdout window (optional)
      - future forecasts

    Key behavior (updated)
    ----------------------
    Changepoints are still selected by holdout MAPE, but the *forecast slope*
    is chosen more safely:
      - Prefer the last-segment slope only if the last segment is long enough
        and not wildly inconsistent with the dominant historical slope.
      - Otherwise, use a damped dominant slope or fall back to a flat (level) forecast.
    """
    site, fuel = str(pdf["Site"].iloc[0]), str(pdf["Fuel"].iloc[0])

    # Ensure a clean, regular weekly series before modeling
    s = pdf.copy()
    s["week_start"] = pd.to_datetime(s["week_start"], errors="coerce")
    s = s.dropna(subset=["week_start"]).sort_values("week_start")

    ts = (
        s.set_index("week_start")["Quantity"]
        .astype(float)
        .asfreq("7D")
        .interpolate("linear")
        .bfill()
        .ffill()
    )
    if len(ts) < 6 or ts.isna().any():
        raise ValueError(f"Insufficient clean training data for {site}-{fuel}: len(ts)={len(ts)}")

    # Holdout is always taken from the most recent observations
    holdout_weeks = int(max(0, holdout_weeks))
    train_end = max(3, len(ts) - holdout_weeks)
    train, holdout = ts.iloc[:train_end], ts.iloc[train_end:]

    # Data-driven floor to prevent negative or collapsing extrapolation
    floor = max(min_floor, float(np.nanpercentile(train.values, floor_percentile)))

    def pack(dates, values, typ, method, conf):
        return pd.DataFrame(
            {
                "Site": site,
                "Fuel": fuel,
                "week_start": dates,
                "value": np.asarray(values, float),
                "type": typ,
                "method": method,
                "confidence_score": conf,
            }
        )

    y = train.values.astype(float)
    x = np.arange(len(train), dtype=float).reshape(-1, 1)

    # Detect candidate changepoints; final selection is done via holdout error
    min_size = int(max(2, min_segment_weeks))
    algo = rpt.Binseg(model="l2", min_size=min_size).fit(y)

    best_mape = np.inf
    best_lr = None
    best_k = 0
    best_bkps = [len(train)]

    # Cap breakpoints to feasible values to avoid ruptures.BadSegmentationParameters
    n = len(train)
    k_feasible_max = max(0, (n // min_size) - 1)
    k_max = 0 if len(holdout) == 0 else min(int(max_breakpoints), k_feasible_max)

    # Evaluate models with 0..K changepoints; scoring uses last-segment extrapolation
    for k in range(0, k_max + 1):
        try:
            bkps = [n] if k == 0 else algo.predict(n_bkps=k)
        except rpt.exceptions.BadSegmentationParameters:
            continue

        seg_start = 0 if len(bkps) == 1 else int(bkps[-2])
        lr = LinearRegression().fit(x[seg_start:], y[seg_start:])

        if len(holdout) == 0:
            best_lr, best_k, best_bkps = lr, k, bkps
            break

        xh = np.arange(len(train), len(train) + len(holdout), dtype=float).reshape(-1, 1)
        pred_h = lr.predict(xh)

        denom = np.where(holdout.values != 0, np.abs(holdout.values), 1.0)
        mape = float(np.mean(np.abs(holdout.values - pred_h) / denom) * 100.0)

        if mape < best_mape:
            best_mape, best_lr, best_k, best_bkps = mape, lr, k, bkps

    # If everything failed for some reason, fall back to a global linear fit
    if best_lr is None:
        best_lr = LinearRegression().fit(x, y)
        best_bkps = [n]
        best_k = 0

    # --- Choose a safe forecast slope (dominant/validated vs last-segment)
    def _segment_slopes(bkps: list[int]) -> list[tuple[int, int, float]]:
        segs = []
        prev = 0
        for b in bkps:
            start, end = prev, int(b)
            if end - start >= 2:
                lr_i = LinearRegression().fit(x[start:end], y[start:end])
                segs.append((start, end, float(lr_i.coef_[0])))
            prev = end
        return segs

    segs = _segment_slopes(best_bkps)
    last_start, last_end, last_slope = segs[-1] if segs else (0, n, float(best_lr.coef_[0]))
    last_len = last_end - last_start

    # Length-weighted “dominant” slope across segments
    if len(segs) >= 2:
        slopes = np.array([s[2] for s in segs], dtype=float)
        lengths = np.array([s[1] - s[0] for s in segs], dtype=float)
        dom_slope = float(np.average(slopes, weights=lengths))
    else:
        dom_slope = float(last_slope)

    slope_eps = 1e-9
    trust_last = (
        (not use_dominant_trend)
        or (
            last_len >= int(min_last_segment_weeks)
            and (np.sign(last_slope) == np.sign(dom_slope) or abs(dom_slope) < slope_eps)
            and (abs(dom_slope) < slope_eps or abs(last_slope) <= 2.0 * abs(dom_slope))
        )
    )

    if trust_last:
        slope = float(last_slope)
        slope_mode = "last"
    else:
        slope = 0.0 if abs(dom_slope) < slope_eps else float(dom_slope)
        slope_mode = "dominant" if slope != 0.0 else "flat"

    # Shrink and cap slope to avoid runaway ramps
    slope = float(slope) * float(np.clip(trend_shrink, 0.0, 1.0))
    if slope_cap is not None:
        cap = float(abs(slope_cap))
        slope = float(np.clip(slope, -cap, cap))

    # Anchor the forecast at the last observed training point
    y_anchor = float(train.iloc[-1])
    t0 = float(n - 1)

    method = f"PiecewiseLinear_{best_k}CP_{slope_mode}"

    # Predict holdout + future using the selected slope
    h_hold, h_fut = len(holdout), int(forecast_horizon)
    steps = np.arange(1, h_hold + h_fut + 1, dtype=float)
    preds = y_anchor + slope * steps  # y(t0 + h) = y(t0) + slope*h

    parts = [pack(train.index, train.values, "actual", "N/A", np.nan)]
    base_conf = float(fallback_confidence)

    if h_hold:
        bt_raw = preds[:h_hold]
        denom = np.where(holdout.values != 0, np.abs(holdout.values), 1.0)
        mape = float(np.mean(np.abs(holdout.values - bt_raw) / denom) * 100.0)
        base_conf = float(np.clip(1.0 - mape / 100.0, 0.0, 1.0))

        bt_vals = np.rint(np.maximum(bt_raw, floor)).astype(int)
        h = np.arange(1, h_hold + 1)
        parts.append(
            pack(
                holdout.index,
                bt_vals,
                "backtest",
                method,
                (base_conf * np.exp(-decay_rate * h)).astype(float),
            )
        )

    fut_index = pd.date_range(ts.index[-1] + pd.Timedelta(days=7), periods=h_fut, freq="7D")
    fut_raw = preds[h_hold:]
    fut_vals = np.rint(np.maximum(fut_raw, floor)).astype(int)

    h = np.arange(1, h_fut + 1)
    parts.append(
        pack(
            fut_index,
            fut_vals,
            "forecast",
            method,
            (base_conf * np.exp(-decay_rate * h)).astype(float),
        )
    )

    return pd.concat(parts, ignore_index=True)


In [ ]:
schema = T.StructType([
    T.StructField("Site", T.StringType()),
    T.StructField("Fuel", T.StringType()),
    T.StructField("week_start", T.TimestampType()),
    T.StructField("value", T.DoubleType()),     # forecast + actual
    T.StructField("type", T.StringType()),      # "actual" or "forecast"
    T.StructField("method", T.StringType()),
    T.StructField("confidence_score", T.DoubleType()),
])

result_df = (
    weekly_full
        .groupBy("Site", "Fuel")
        .applyInPandas(
            lambda pdf: fit_and_forecast_ets(
                pdf,
                holdout_weeks=holdout_weeks,
                forecast_horizon=forecast_horizon,
            ),
            schema=schema,
        )
)


## Write Forecasts
Clean-up Past Forecasting Runs and write to `ForecastWeekly`

In [ ]:
%%sql
drop table ForecastWeekly

In [ ]:
table_name = "ForecastWeekly"  # your downstream table

# Overwrite existing table with the new forecast
result_df.write.format("delta")\
                  .mode("overwrite")\
                  .saveAsTable(table_name)

print(f" Forecasts written to Fabric data table: {table_name}")

In [ ]:
# Read the 'forecastweekly' table from the Lakehouse
df = spark.read.table("forecastweekly")

# Select top 1000 rows
top_1000_df = df.limit(1000)

# Display the result (if desired)
display(top_1000_df)

| Confidence Score | Meaning                                             |
|------------------|-----------------------------------------------------|
| **0.85–1.00**    | Very high confidence. Model consistently accurate.  |
| **0.65–0.85**    | Moderate confidence. Forecast is directional but has noise. |
| **0.40–0.65**    | Low confidence. Forecast is uncertain.              |
| **< 0.40**       | Very low confidence. Should not be used for decisions. |


In [ ]:
site = ""
fuel = ""

filtered = (
    df.filter((df.Site == site) & (df.Fuel == fuel))
      .orderBy("week_start")
)

pdf = filtered.toPandas()
display(pdf)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(14,6))

# Actuals
actual = pdf[pdf["type"] == "actual"]
plt.plot(actual["week_start"], actual["value"], label="Actual", marker="o")

# Backtest
backtest = pdf[pdf["type"] == "backtest"]
plt.plot(backtest["week_start"], backtest["value"], label="Backtest", linestyle="--")

# Forecast
forecast = pdf[pdf["type"] == "forecast"]
plt.plot(forecast["week_start"], forecast["value"], label="Forecast", marker="x")

# CONFIDENCE RIBBON
conf = forecast["confidence_score"].values        # 0 → 1
vals = forecast["value"].values.astype(float)

# Convert confidence into a relative uncertainty width
uncertainty = (1 - conf) * vals * 0.3  
# 0.3 = scaling factor; for band width

upper = vals + uncertainty
lower = vals - uncertainty

plt.fill_between(
    forecast["week_start"],
    lower,
    upper,
    color="orange",
    alpha=0.20,
    label="Forecast Uncertainty"
)

plt.title(f"Actual vs Forecast — {site} / {fuel}")
plt.xlabel("Week")
plt.ylabel("Value")
plt.legend()
plt.grid(True)
plt.show()

## Steps to Improve Forecast Accuracy

1. **Clean and prepare the data**
   - Fix missing dates and enforce a consistent weekly frequency.
   - Correct or remove negative or impossible values.
   - Smooth noisy series using moving averages if appropriate.

2. **Ensure enough historical data**
   - Models perform better with longer histories.
   - Aim for at least 1–2 years of weekly data when possible.

3. **Improve seasonality detection**
   - Use seasonal models only when seasonality is real.
   - Tune or disable seasonality for flat or irregular series.

4. **Use a more robust validation method**
   - Increase the holdout window (e.g., 8–12 weeks instead of 4).
   - Use multiple backtesting windows for more stable accuracy metrics.

5. **Handle low-volume or erratic series separately**
   - Use simpler models (e.g., moving average, last value).
   - Avoid ETS on sparse or highly volatile data.

6. **Remove abnormal events from training**
   - Exclude or adjust for unusual spikes, outages, promotions, or data errors.
